# 💡 QLoRA дообучение DeepSeek 14B Chat на своих файлах

In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes trl sentencepiece python-docx --quiet


## 📁 Конвертация .txt и .docx в JSONL

In [ ]:
import os
import json
from docx import Document

input_dir = "your_files"  # Папка с твоими .txt и .docx
output_path = "data.jsonl"

os.makedirs(input_dir, exist_ok=True)

def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join(p.text for p in doc.paragraphs if p.text.strip())

with open(output_path, "w", encoding="utf-8") as out:
    for fname in os.listdir(input_dir):
        path = os.path.join(input_dir, fname)
        if fname.endswith(".txt"):
            text = open(path, encoding="utf-8").read().strip()
        elif fname.endswith(".docx"):
            text = read_docx(path)
        else:
            continue

        if text:
            json.dump({"text": text}, out, ensure_ascii=False)
            out.write("\n")

print("✅ Данные сохранены в data.jsonl")


## 🧠 Обучение модели с QLoRA

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset

model_name = "deepseek-ai/deepseek-llm-14b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype="bfloat16")
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=bnb_config, trust_remote_code=True)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

dataset = load_dataset("json", data_files="data.jsonl", split="train")

def tokenize(sample):
    return tokenizer(sample["text"], truncation=True, padding="max_length", max_length=2048)

tokenized = dataset.map(tokenize, batched=True)

training_args = TrainingArguments(
    output_dir="./deepseek-lora-chat",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=2,
    bf16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


## 🔗 Слияние весов LoRA с базовой моделью

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

peft_model_id = "./deepseek-lora-chat"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto", trust_remote_code=True)
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.merge_and_unload()

model.save_pretrained("./merged-model")
tokenizer.save_pretrained("./merged-model")
print("✅ Слиты и сохранены в ./merged-model")


## ✅ Готово! Можно конвертировать в GGUF отдельно для Ollama

## 🧱 Экспорт модели в GGUF для использования в Ollama

In [ ]:
# Убедись, что у тебя установлен llama.cpp
# https://github.com/ggerganov/llama.cpp

# 1. Клонируем репозиторий и собираем llama.cpp
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!make -j

# 2. Запускаем конвертацию из HuggingFace в GGUF
# Путь до модели — ../merged-model (относительно llama.cpp)
!python3 convert.py --outfile ../deepseek-merged.gguf --model-dir ../merged-model

# 3. Файл deepseek-merged.gguf готов для использования в Ollama
